#### Scripts

##### Import

In [363]:
import sys,os
import datetime, time
import cx_Oracle
import pandas as pd
import numpy as np
import traceback
import logging
import pandas.io.formats.excel


BASE_DIR = r"C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test"
sys.path.append(BASE_DIR)

from string import Template

from core import sdm_check
from core.configadmin import ConfigAdmin
from core.color_me import ColorMe
from core.outlook import send_email
from core import log_ctrl
from xlsxwriter.workbook import Workbook
from xlsxwriter.worksheet import Worksheet
# from StyleFrame import StyleFrame

##### Config

In [366]:
# TODO:Config
config_path = os.path.join(BASE_DIR, 'conf')
config_name = 'setting.ini'
conf = ConfigAdmin(config_path, config_name)
# Template
MyTemp = os.path.join (config_path, 'CSS', conf.read_config('CSS_Style', 'CRMCampaignTemplate'))
# Log Info
log_info = log_ctrl.de8ug_log(logger_name='CampaignReadRateReprot', log_file=os.path.join(BASE_DIR, 'log', f'{time.strftime("%Y%m",time.localtime())}-monitor.log'), level=logging.INFO)
# SQL Script
CampaignListWaitToCheck = os.path.join(BASE_DIR, 'db','sql','CampaignListWaitToCheck.sql')
CampaignAnaylysis = os.path.join(BASE_DIR, 'db','sql','CampaignAnaylysis.sql')
# SAS DB
user = conf.read_config("SAS_db", "crm_user")
password = conf.read_config("SAS_db", "crm_password")
service = conf.read_config("SAS_db", "crm_service")
# panda option
pd.set_option("display.max_columns",None)
pd.set_option('display.max_rows',None)   #显示所有行
pd.set_option('max_colwidth',200)     #设置数据的显示长度，默认为50
pd.set_option('display.width', 25)    # 设置行宽
pd.set_option('colheader_justify', 'center')    #设置居中
pd.set_option('expand_frame_repr', True)     ##禁止自动换行(设置为Flase不自动换行，True反之)
FloatFormat = lambda x: format(x, '.2%') if x != np.nan else ""
# E-mail
SendTo = conf.read_config('E-mail', 'SenderOfCrmReport')
CopyTo = conf.read_config('E-mail', 'CopyerOfCrmReport')

##### Class

In [460]:
class CampAnalysisToCrm(object):
    def __init__(self, frequncy):
        self.db = cx_Oracle.connect(user, password, service)
        self.con = self.db.cursor()
        self.sqlList = {}
        self.frequncy = str.capitalize(frequncy)
        self.FileExcel = os.path.join(conf.read_config('Cofig_Path','D3_Report'),f'CRM {self.frequncy} Campaign Report {time.strftime("%Y%m%d",time.localtime())}.xlsx')
        if self.frequncy == 'Daily':
            self.sqlList['Push'] = open(os.path.join(BASE_DIR, "db", "sql", "Daily_PUSH_CRM.sql"),'r',encoding="utf-8").read()
            self.sqlList['SMS'] = open(os.path.join(BASE_DIR, "db", "sql", "Daily_SMS_CRM.sql"),'r',encoding="utf-8").read()
            self.sqlList['Wechat'] = open(os.path.join(BASE_DIR, "db", "sql", "Daily_Wechat_CRM.sql"),'r',encoding="utf-8").read()
        elif self.frequncy == 'Weekly':
            self.sqlList['Push'] = open(os.path.join(BASE_DIR, "db", "sql", "Weekly_PUSH_CRM.sql"),'r',encoding="utf-8").read()
            self.sqlList['SMS'] = open(os.path.join(BASE_DIR, "db", "sql", "Weekly_SMS_CRM.sql"),'r',encoding="utf-8").read()
            self.sqlList['Wechat'] = open(os.path.join(BASE_DIR, "db", "sql", "Weekly_Wechat_CRM.sql"),'r',encoding="utf-8").read()
        else:
            raise Exception('Wrong frequncy, Pleas Check.')
        
        self.writer = pd.ExcelWriter(self.FileExcel)
        self.sender = conf.read_config('E-mail', 'CampAnalyCRM_To')
        self.copyer = conf.read_config('E-mail', 'CampAnalyCRM_Cc')
        
    def LoadData(self):
        try:
            self.DataList = {}
            for k,v in self.sqlList.items():
                log_info.info(f'Start to loading {k} data')
                row = self.con.execute(v).fetchall()
                cols = [i[0] for i in self.con.description]
                df = pd.DataFrame(data=row, index = [i+1 for i in range(len(row))], columns=cols)
                if len(row) > 0:
                    Dlist = []
                    Dlist.append(df)
                    res = self.GetNum(k, df)
                    if res != False:
                        Dlist = Dlist + res
                        log_info.info(f'{k} data loading finished.')
                    else:
                        print(ColorMe("Warn:").yellow(), 'Process has been breaked!')
                        break
                    self.DataList[k] = Dlist
                else:
                    log_info.info("No result:", v)
                    print(ColorMe("Warn:").yellow(), f'{k} has no results.')
                    
            print(ColorMe("Succ:").green(), 'All data Loading Completed.')
            self.CloseDB()
            return self.DataList
        except Exception as e:
            print(ColorMe("Error:").red(), e)
            log_info.error(traceback.format_exc())
            return False
    
    def GetNum(self, channel, data):
        try:
            a = data['SUCCESS_RATE'].apply(lambda x: float(str(x).replace("%","")))
            b = []
            res = []
            if channel == "Push":
                b = list(a.loc[a.values<30].index)
            elif channel == 'SMS':
                b = list(a.loc[a.values<78].index)
            elif channel == 'Wechat':
                b = list(a.loc[a.values<100].index)
            
            if self.frequncy == 'Weekly':
                c = data.loc[data.index.isin(b)].sort_values(by=['PRODUCT', 'SEND_DATE', 'SUCCESS_RATE'])
            else:
                c = data.loc[data.index.isin(b)].sort_values(by=['EXECUTED_DATE'])
            res.append(len(c))
            res.append(c.to_html(index=False, border=1, classes='f1-table'))
            return res
        except Exception as e:
            print(ColorMe("Error:").red(), e)
            log_info.error(traceback.format_exc())
            return False
    
    def SaveToExcel(self):
        self.content = "<p> The inspection results are as follows:</p>"
        for k,v in self.DataList.items():
            if v[1] != 0:
                self.content = self.content + f"<p>{k} has {v[1]} abnormal success rate of campaign.<p> {v[2]}"
                                          
            df = v[0]
            df['SEND_DATE'] = df['SEND_DATE'].apply(lambda x: datetime.datetime.strftime(x,"%Y-%m-%d"))
            #Get the Statistics Data.
            gp = df.groupby(['SEND_DATE'])['CNT_SUCCESS','SEND_TOTAL'].sum().reset_index()
            df = df.drop(['CNT_SUCCESS'], axis=1)
            df.to_excel(self.writer, k, startrow=1, index=0, header=False)
            
            # Get the xlsxwriter workbook and worksheet objects.
            workbook = self.writer.book
            worksheets = self.writer.sheets
            worksheet = worksheets[k]
            
            # Weekly add a Chart
            if self.frequncy == 'Weekly':
                bold = workbook.add_format({'bold': 1})
                date_format = workbook.add_format({'num_format': 'dd/mm/yyyy'})
                gp['SUCCESS_RATE'] = gp.apply(lambda x: x['CNT_SUCCESS']/x['SEND_TOTAL'], axis=1)
                gp = gp.drop(['CNT_SUCCESS','SEND_TOTAL'], axis=1)
                # Add the worksheet data that the charts will refer to.
                headings = list(gp.columns)
                # Write the Statistics Data.                  
                worksheet.write_row('S1', headings, bold)
                worksheet.write_column('S2', gp['SEND_DATE'], date_format)
                worksheet.write_column('T2', gp['SUCCESS_RATE'])
                # Create a new chart object. In this case an embedded chart.
                chart1 = workbook.add_chart({'type': 'line'})
                # Configure the first series.
                chart1.add_series({'name':f'={k}!$T$1',
                                   'categories': f'={k}!$S$2:$S${len(gp)+1}',
                                   'values':f'={k}!$T$2:$T${len(gp)+1}',})
                # Add a chart title and some axis labels.
                chart1.set_title ({'name': f'Success Rate Of {k}'})
                chart1.set_x_axis({'name': 'Send Date'})
                chart1.set_y_axis({'name': 'Success Rate'})

                # Set an Excel chart style. Colors with white outline and shadow.
                chart1.set_style(10)

                # Insert the chart into the worksheet (with an offset).
                worksheet.insert_chart('G1', chart1, {'x_offset': 25, 'y_offset': 10})
                                      
            # set column width
            worksheet.set_column("A:C", 18)
            worksheet.set_column("D:D", 60)
            worksheet.set_column("E:F", 25)
            # Set Format
            WorningFormat = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'}) # Warning Format
            HeaderFormat = workbook.add_format({'bold':  True, 'align': 'center', 'valign': 'center', 'text_wrap': True, 'bg_color': '#5B9BD5', 'font_color': '#FFFFFF'})
            # Set Head Format
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, HeaderFormat)
            worksheet.conditional_format('B2:B9999',{'type':'formula','criteria': '=ROUND(MID($B2,1,FIND("%",$B2)-1),2)<=30', 'format':WorningFormat})
                                     
        self.writer.save()
        self.writer.close()
    
    def CloseDB(self):
        self.db.close()
    
    def SendEmail(self):
        self.subject = f'CRM {self.frequncy} Campaign Report {time.strftime("%Y%m%d",time.localtime())}'
        with open(MyTemp, 'r') as f:
            content = Template(f.read()).substitute(content = self.content).replace('dataframe f1-table','fl-table')
        res = send_email.outlook(send_to=self.sender, subject=self.subject, content=content, cc_to=self.copyer, addtach=self.FileExcel)
        if not res:
            log_info.error('E-mail Send Faild!')
            print(ColorMe(f'{datetime.datetime.now().strftime("%Y_%m_%d %X")} E-mail Send Faild!').red())
            return False
        else:
            log_info.info('E-mail Send Success!')
            print(ColorMe(f'{datetime.datetime.now().strftime("%Y_%m_%d %X")} E-mail Send Success!').green())
            return True

#### Test

##### Main

In [461]:
def main(frequncy):
    status = True
    try:
        job = CampAnalysisToCrm(frequncy)
        job.LoadData()
        job.SaveToExcel()
        job.SendEmail()
    except Exception as e:
        print("Something is wrong!",traceback.format_exc())
        status = False
    finally:
        return status
        print("Exit the Program.")

In [463]:
if __name__ == "__main__":
    try:
        print(ColorMe(f'Start CRM-Campaign Check Programe - {os.getpid()}').blue())
        status = main('weekly')
        if status:
            log_info.debug('Programe Success!')
        else:
            log_info.warning('Programe False!')
            raise Exception('Unknown Error, programe false!')
    except Exception as e:
        log_info.critical(traceback.format_exc())
        print(ColorMe("Error:").red(),e)
        # subject = "XRM D-3 Campaign Monitor"
        # send_email.outlook(send_to=Sender, subject=subject, content=content, cc_to=None, addtach=None)
        os.system("pause")
    finally:
        print(ColorMe(f'Exit Programe {os.getpid()}').blue())

Start CRM-Campaign Check Programe - 15856


2020-03-03 12:22:36,119 <ipython-input-460-d7ff3d8db116> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 12:22:36,119 <ipython-input-460-d7ff3d8db116> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 12:22:36,119 <ipython-input-460-d7ff3d8db116> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 12:22:36,119 <ipython-input-460-d7ff3d8db116> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 12:22:36,119 <ipython-input-460-d7ff3d8db116> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 12:22:36,119 <ipython-input-460-d7ff3d8db116> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 12:22:36,119 <ipython-input-460-d7ff3d8db116> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 12:22:36,119 <ipython-input-460-d7ff3d8db116> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 12:22:36,119 

2020-03-03 12:36:02,475 <ipython-input-460-d7ff3d8db116> [line:37] CampaignReadRateReprot INFO Wechat data loading finished.
2020-03-03 12:36:02,475 <ipython-input-460-d7ff3d8db116> [line:37] CampaignReadRateReprot INFO Wechat data loading finished.
2020-03-03 12:36:02,475 <ipython-input-460-d7ff3d8db116> [line:37] CampaignReadRateReprot INFO Wechat data loading finished.
2020-03-03 12:36:02,475 <ipython-input-460-d7ff3d8db116> [line:37] CampaignReadRateReprot INFO Wechat data loading finished.
2020-03-03 12:36:02,475 <ipython-input-460-d7ff3d8db116> [line:37] CampaignReadRateReprot INFO Wechat data loading finished.


Succ: All data Loading Completed.


2020-03-03 12:36:07,654 <ipython-input-460-d7ff3d8db116> [line:151] CampaignReadRateReprot INFO E-mail Send Success!
2020-03-03 12:36:07,654 <ipython-input-460-d7ff3d8db116> [line:151] CampaignReadRateReprot INFO E-mail Send Success!
2020-03-03 12:36:07,654 <ipython-input-460-d7ff3d8db116> [line:151] CampaignReadRateReprot INFO E-mail Send Success!
2020-03-03 12:36:07,654 <ipython-input-460-d7ff3d8db116> [line:151] CampaignReadRateReprot INFO E-mail Send Success!
2020-03-03 12:36:07,654 <ipython-input-460-d7ff3d8db116> [line:151] CampaignReadRateReprot INFO E-mail Send Success!
2020-03-03 12:36:07,654 <ipython-input-460-d7ff3d8db116> [line:151] CampaignReadRateReprot INFO E-mail Send Success!
2020-03-03 12:36:07,654 <ipython-input-460-d7ff3d8db116> [line:151] CampaignReadRateReprot INFO E-mail Send Success!
2020-03-03 12:36:07,654 <ipython-input-460-d7ff3d8db116> [line:151] CampaignReadRateReprot INFO E-mail Send Success!
2020-03-03 12:36:07,654 <ipython-input-460-d7ff3d8db116> [line:1

2020_03_03 12:36:07 E-mail Send Success!
Exit Programe 15856


##### Debug

###### Funtiong

In [457]:
test = CampAnalysisToCrm("weekly")

In [458]:
data = test.LoadData()

2020-03-03 11:42:30,043 <ipython-input-456-070b3dae0e19> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 11:42:30,043 <ipython-input-456-070b3dae0e19> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 11:42:30,043 <ipython-input-456-070b3dae0e19> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 11:42:30,043 <ipython-input-456-070b3dae0e19> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 11:42:30,043 <ipython-input-456-070b3dae0e19> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 11:42:30,043 <ipython-input-456-070b3dae0e19> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 11:42:30,043 <ipython-input-456-070b3dae0e19> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 11:42:30,043 <ipython-input-456-070b3dae0e19> [line:27] CampaignReadRateReprot INFO Start to loading Push data
2020-03-03 11:42:30,043 

2020-03-03 11:52:07,924 <ipython-input-456-070b3dae0e19> [line:37] CampaignReadRateReprot INFO Wechat data loading finished.
2020-03-03 11:52:07,924 <ipython-input-456-070b3dae0e19> [line:37] CampaignReadRateReprot INFO Wechat data loading finished.
2020-03-03 11:52:07,924 <ipython-input-456-070b3dae0e19> [line:37] CampaignReadRateReprot INFO Wechat data loading finished.
2020-03-03 11:52:07,924 <ipython-input-456-070b3dae0e19> [line:37] CampaignReadRateReprot INFO Wechat data loading finished.
2020-03-03 11:52:07,924 <ipython-input-456-070b3dae0e19> [line:37] CampaignReadRateReprot INFO Wechat data loading finished.


Succ: All data Loading Completed.


In [459]:
test.SaveToExcel()
# df = data['Push'][0]
# df['SEND_DATE'] = df['SEND_DATE'].apply(lambda x: datetime.datetime.strftime(x,"%Y-%m-%d"))

In [329]:
test.SendEmail()
# test.content

2020-02-27 16:25:03,378 <ipython-input-325-c6927cfd81c8> [line:112] CampaignReadRateReprot INFO E-mail Send Success!
2020-02-27 16:25:03,378 <ipython-input-325-c6927cfd81c8> [line:112] CampaignReadRateReprot INFO E-mail Send Success!
2020-02-27 16:25:03,378 <ipython-input-325-c6927cfd81c8> [line:112] CampaignReadRateReprot INFO E-mail Send Success!
2020-02-27 16:25:03,378 <ipython-input-325-c6927cfd81c8> [line:112] CampaignReadRateReprot INFO E-mail Send Success!
2020-02-27 16:25:03,378 <ipython-input-325-c6927cfd81c8> [line:112] CampaignReadRateReprot INFO E-mail Send Success!
2020-02-27 16:25:03,378 <ipython-input-325-c6927cfd81c8> [line:112] CampaignReadRateReprot INFO E-mail Send Success!
2020-02-27 16:25:03,378 <ipython-input-325-c6927cfd81c8> [line:112] CampaignReadRateReprot INFO E-mail Send Success!
2020-02-27 16:25:03,378 <ipython-input-325-c6927cfd81c8> [line:112] CampaignReadRateReprot INFO E-mail Send Success!
2020-02-27 16:25:03,378 <ipython-input-325-c6927cfd81c8> [line:1

2020_02_27 16:25:03 E-mail Send Success!


True

###### ExcelWriter

In [373]:
test.writer = pd.ExcelWriter(test.FileExcel)

In [296]:
# df['SEND_DATE'] = df['SEND_DATE'].apply(lambda x: datetime.datetime.strftime(x,"%Y-%m-%d"))
df.to_excel(test.writer, 'Push', startrow=1, index=0, header=False)
workbook = test.writer.book
worksheets = test.writer.sheets
worksheet = worksheets['Push']
# set column width
worksheet.set_column("A:C", 18)
worksheet.set_column("D:D", 60)
worksheet.set_column("E:F", 25)
# Set Head Format
for col_num, value in enumerate(df.columns.values):
    worksheet.write(0, col_num, value, HeaderFormat)
worksheet.conditional_format('B2:B9999',{'type':'formula','criteria': '=ROUND(MID($B2,1,FIND("%",$B2)-1),2)<=30', 'format':WorningFormat})

In [297]:
test.writer.save()
test.writer.close()

###### Picture

In [420]:
df = data['Push'][0]
# data['Push'][0]

In [434]:
# df['SUCCESS_RATE'] = df['SUCCESS_RATE'].apply(lambda x: float(x.replace("%","")/100))
gp = df.groupby(['SEND_DATE'])['CNT_SUCCESS','SEND_TOTAL'].sum().reset_index()
gp['SUCCESS_RATE'] = gp.apply(lambda x: x['CNT_SUCCESS']/x['SEND_TOTAL'], axis=1)
gp = gp.drop(['CNT_SUCCESS','SEND_TOTAL'], axis=1)

In [449]:
# df
list(gp.columns)

['SEND_DATE', 'SUCCESS_RATE']

##### Memo

###### Prepare

In [99]:
db = cx_Oracle.connect(user, password, service)
con = db.cursor()

In [100]:
# TODO
# For daily monitor, please send result from 11AM to 19PM every day. 
# For weekly monitor, please execute and send result every day on 9AM.
DailyPush = open(os.path.join(BASE_DIR, "db", "sql", "Daily_PUSH_CRM.sql"),'r',encoding="utf-8").read()
DailySMS =  open(os.path.join(BASE_DIR, "db", "sql", "Daily_SMS_CRM.sql"),'r',encoding="utf-8").read()
DailyWechat =  open(os.path.join(BASE_DIR, "db", "sql", "Daily_Wechat_CRM.sql"),'r',encoding="utf-8").read()
WeeklyPush = open(os.path.join(BASE_DIR, "db", "sql", "Weekly_PUSH_CRM.sql"),'r',encoding="utf-8").read()
WeeklySMS =  open(os.path.join(BASE_DIR, "db", "sql", "Weekly_SMS_CRM.sql"),'r',encoding="utf-8").read()
WeeklyWechat =  open(os.path.join(BASE_DIR, "db", "sql", "Weekly_Wechat_CRM.sql"),'r',encoding="utf-8").read()

###### GetData

In [232]:
row = con.execute(DailyPush).fetchall()
cols = [i[0] for i in con.description]
df = pd.DataFrame(data=row, index = [i+1 for i in range(len(row))], columns=cols)

In [333]:
df

,SEND_DATE,SUCCESS_RATE,PRODUCT,CAMPAIGN_NAME,PUSH_EXECUTED_DATE,SEND_TOTAL
1,2020-02-27,39.1%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-27 14:08:30,353
2,2020-02-27,46.8%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-27 13:25:37,878
3,2020-02-27,48.6%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-27 14:36:24,749
4,2020-02-27,51.1%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-27 14:17:38,225
5,2020-02-27,57.4%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-27 14:03:29,709
6,2020-02-27,63.9%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-27 13:14:17,166
7,2020-02-27,64.9%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-27 14:28:49,188
8,2020-02-27,58.7%,HCP,HCP_Engagement_Minimum Repay_Campaign,2020-02-27 12:58:33,1007
9,2020-02-27,32.9%,HCP,HCP_Followup_AF_CAPP,2020-02-27 12:47:21,568
10,2020-02-27,45.1%,HCP,HCP_Onboarding_T+12_Cashback_Daily,2020-02-27 12:14:04,273


In [353]:
a = df['SUCCESS_RATE'].apply(lambda x: float(str(x).replace("%","")))
b = list(a.loc[a.values<30].index)
c = df.loc[df.index.isin(b)]
num = len(c)

In [358]:
html = c.to_html(border=1, classes='f1-table')
print(html.replace('dataframe fl-table','fl-table'))

<table border="1" class="dataframe f1-table">
  <thead>
    <tr style="text-align: center;">
      <th></th>
      <th>SEND_DATE</th>
      <th>SUCCESS_RATE</th>
      <th>PRODUCT</th>
      <th>CAMPAIGN_NAME</th>
      <th>PUSH_EXECUTED_DATE</th>
      <th>SEND_TOTAL</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>11</th>
      <td>2020-02-27</td>
      <td>26.5%</td>
      <td>HCP</td>
      <td>HCP_Onboarding_T+1_Cashback_Daily</td>
      <td>2020-02-27 11:47:35</td>
      <td>878</td>
    </tr>
    <tr>
      <th>15</th>
      <td>2020-02-27</td>
      <td>0%</td>
      <td>Other</td>
      <td>Cash Loan Evidence CAPP - 6</td>
      <td>2020-02-27 15:13:02</td>
      <td>4021</td>
    </tr>
    <tr>
      <th>16</th>
      <td>2020-02-27</td>
      <td>0%</td>
      <td>Other</td>
      <td>Cash Loan Evidence CAPP - 7</td>
      <td>2020-02-27 15:25:29</td>
      <td>5574</td>
    </tr>
    <tr>
      <th>17</th>
      <td>2020-02-27</td>
      <td>0%</td>
      <td>Other</t

###### Write to Excel

In [198]:
writer = pd.ExcelWriter(D3_excel)
# df['SUCCESS_RATE'] = df['SUCCESS_RATE'].apply(lambda x: format(x, '.2%'))
# df['PUSH_START_DATE'] = pd.to_datetime(df.PUSH_START_DATE, format="%Y/%m/%d")
df['PUSH_START_DATE'] = df['PUSH_START_DATE'].apply(lambda x: datetime.datetime.strftime(x,"%Y-%m-%d"))
# df[['PUSH_START_DATE']] = df[['PUSH_START_DATE']].astype('object')
# df['PUSH_START_DATE'] = df['PUSH_START_DATE'].apply(str)
df.to_excel(writer, 'Push', startrow=1, index=0, header=False)

In [445]:
import xlsxwriter

workbook = xlsxwriter.Workbook('chart_line.xlsx')
worksheet = workbook.add_worksheet()
bold = workbook.add_format({'bold': 1})
date_format = workbook.add_format({'num_format': 'dd/mm/yyyy'})
# Add the worksheet data that the charts will refer to.
headings = ['SEND_DATE', 'SUCCESS_RATE']
data = [
    [2, 3, 4, 5, 6, 7],
    [10, 40, 50, 20, 10, 50],
    [30, 60, 70, 50, 40, 30],
]

worksheet.write_row('A1', headings, bold)
worksheet.write_column('A2', gp['SEND_DATE'], date_format)
worksheet.write_column('B2', gp['SUCCESS_RATE'])
# worksheet.write_column('C2', data[2])

# Create a new chart object. In this case an embedded chart.
chart1 = workbook.add_chart({'type': 'line'})

# Configure the first series.
chart1.add_series({
    'name':       '=Sheet1!$B$1',
    'categories': '=Sheet1!$A$2:$A$9',
    'values':     '=Sheet1!$B$2:$B$9',
})

# Configure second series. Note use of alternative syntax to define ranges.
# chart1.add_series({
#     'name':       ['Sheet1', 0, 2],
#     'categories': ['Sheet1', 1, 0, 6, 0],
#     'values':     ['Sheet1', 1, 2, 6, 2],
# })

# Add a chart title and some axis labels.
chart1.set_title ({'name': 'Results of sample analysis'})
chart1.set_x_axis({'name': 'Test number'})
chart1.set_y_axis({'name': 'Sample length (mm)'})

# Set an Excel chart style. Colors with white outline and shadow.
chart1.set_style(10)

# Insert the chart into the worksheet (with an offset).
worksheet.insert_chart('D2', chart1, {'x_offset': 25, 'y_offset': 10})


workbook.close()

In [415]:
os.getcwd()

'C:\\Users\\leo.zhangzs\\AppData\\Roaming\\Code\\User\\VScode\\base'

###### Format Config

In [199]:
# Get the xlsxwriter workbook and worksheet objects.
workbook1 = writer.book
worksheets = writer.sheets
worksheet1 = worksheets['Push']

In [200]:
# Set Format
WorningFormat = workbook1.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'}) # Warning Format
HeaderFormat = workbook1.add_format({'bold':  True, 'align': 'center', 'valign': 'center', 'text_wrap': True, 'bg_color': '#5B9BD5', 'font_color': '#FFFFFF'})
# set column width
worksheet1.set_column("A:C", 18)
worksheet1.set_column("D:D", 60)
worksheet1.set_column("E:F", 25)
# Set Head Format
# worksheet1.set_row(0, cell_format=format2)
# Write the column headers with the defined format.
for col_num, value in enumerate(df.columns.values):
    worksheet1.write(0, col_num, value, HeaderFormat)

worksheet1.conditional_format('B2:B9999',{'type':'formula','criteria': '=ROUND(MID($B2,1,FIND("%",$B2)-1),2)<=30', 'format':WorningFormat})

###### Save To Excel 

In [201]:
writer.save()
writer.close()

In [233]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 1 to 18
Data columns (total 6 columns):
PUSH_START_DATE       18 non-null datetime64[ns]
SUCCESS_RATE          18 non-null object
PRODUCT               18 non-null object
CAMPAIGN_NAME         18 non-null object
PUSH_EXECUTED_DATE    18 non-null datetime64[ns]
SEND_TOTAL            18 non-null int64
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 792.0+ bytes


,PUSH_START_DATE,SUCCESS_RATE,PRODUCT,CAMPAIGN_NAME,PUSH_EXECUTED_DATE,SEND_TOTAL
1,2020-02-27,0%,HCP,HCP_Followup_AF_CAPP,2020-02-27 12:47:21,568
2,2020-02-27,0%,HCP,HCP_Onboarding_T+12_Cashback_Daily,2020-02-27 12:14:04,273
3,2020-02-27,0%,HCP,HCP_Onboarding_T+1_Cashback_Daily,2020-02-27 11:47:35,878
4,2020-02-27,0%,HCP,HCP_Onboarding_T+5_Cashback_Daily,2020-02-27 11:09:45,711
5,2020-02-27,41.7%,HCP,RTDM_AF_FUP,2020-02-27 09:56:26,223
6,2020-02-27,72.8%,HCP,RTDM_ORBP_AF_FUP,2020-02-27 10:25:07,184
7,2020-02-27,0%,Other,Reminding Capp,2020-02-27 12:21:34,159
8,2020-02-27,0%,RPOS,0 MOB Rpos_0_sms_push,2020-02-27 12:13:52,376
9,2020-02-27,0%,RPOS,0 MOB Rpos_0_sms_push,2020-02-27 12:22:23,809
10,2020-02-27,0%,RPOS,Daily_CAPP_After_Scoring_Rpos,2020-02-27 12:09:00,2416


In [242]:
a = df['SUCCESS_RATE'].apply(lambda x: float(str(x).replace("%","")))
len(a.loc[a.values<=30])

12

##### Sent Email

In [19]:
subject = 'Test'
send_to = "Leo.ZhangZS@homecredit.cn"
subject = 'Test Message'
# img = r'C:\Users\leo.zhangzs\Documents\HCC\Campagin\Analysis\SA ACL individual task Result Notice success SCR-764.png'
# with open(img, "rb") as f:
#     base64_data = base64.b64encode(f.read())
# content = f'<img alt="" src="data:image/png;base64,{base64_data}">'
hh = r"C:\Users\leo.zhangzs\Downloads\jq228985.html"
with open(hh, 'r', encoding='utf-8') as f:
    content = f.read()
# attach = r'C:\Users\leo.zhangzs\Documents\HCC\Campagin\Analysis\Tipper 2nd ACL Lucky Draw have contract group SCR-701.html'
# patt = []
# patt.append(img)
# patt.append(attach)

In [20]:
send_email.outlook(send_to,subject,content,addtach = None)
print("send email ok!!!!!!!!!!")

send email ok!!!!!!!!!!


##### Pyecharts

In [454]:
df

,SEND_DATE,SUCCESS_RATE,PRODUCT,CAMPAIGN_NAME,EXECUTED_DATE,CNT_SUCCESS,SEND_TOTAL
1,2020-02-24,33.8%,HCP,1_HCP_CAPP_part1,2020-02-22 18:58:49,13098,38744
2,2020-02-24,62.1%,HCP,Daily_HCP_Contract_Confirmation,2020-02-24 17:57:10,64,103
3,2020-02-24,47.6%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-24 15:16:18,121,254
4,2020-02-24,51.5%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-24 15:28:43,397,771
5,2020-02-24,52.7%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-24 16:26:54,176,334
6,2020-02-24,53.2%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-24 16:38:17,118,222
7,2020-02-24,53.4%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-24 16:54:50,110,206
8,2020-02-24,57.4%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-24 16:18:55,178,310
9,2020-02-24,57.5%,HCP,HCP Migration Education Campaign_new homepage_daily,2020-02-24 17:04:42,462,803
10,2020-02-24,46.4%,HCP,HCP_Followup_AF_CAPP,2020-02-24 14:59:45,267,575
